In [127]:
import pandas as pd
import html5lib
import re
import pyodbc
from datetime import datetime
import logging
from bs4 import BeautifulSoup
import sys
import requests as req
import io
from PIL import Image
from io import BytesIO

In [148]:
#url = 'https://www.sport-express.ru/football/L/command/78/players/'
url = 'https://www.sport-express.ru/football/L/command/62/players/'
resp = req.get(url) 
soup_team = BeautifulSoup(resp.text, 'lxml')

In [142]:
print(soup_team.prettify())

<!DOCTYPE html>
<html>
 <head>
  <!-- TemplateTag : football -->
  <script type="text/javascript">
   var TemplateTag = 'football';
  </script>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="a6ff411fa6eb291e" name="yandex-verification"/>
  <link href="/manifest.json" rel="manifest"/>
  <meta content="kLTDoWD5oaJc9lp-iGXHnN5eyOSaTaevi-t-8lnfno8" name="google-site-verification"/>
  <link href="//ss.sport-express.ru/img/favs/apple/apple-touch-icon-57x57.png" rel="apple-touch-icon" sizes="57x57" type="image/png"/>
  <link href="//ss.sport-express.ru/img/favs/apple/apple-touch-icon-60x60.png" rel="apple-touch-icon" sizes="60x60" type="image/png"/>
  <link href="//ss.sport-express.ru/img/favs/apple/apple-touch-icon-72x72.png" rel="apple-touch-icon" sizes="72x72" type="image/png"/>
  <link href="//ss.sport-express.ru/img/favs/apple/apple-touch-icon-76x76.png" rel="apple-touch-icon" sizes="76x76" 

In [149]:
player_table = soup_team.find("table", {"class": "se19-table"})

In [150]:
type(player_table)

bs4.element.Tag

In [151]:
player_list = player_table.find('tbody').find_all('tr')
#player_list = player_table.find_all('tbody')

In [138]:
for tr in player_list:
    #print(tr)
    tr.find_all('td')

In [152]:
#, columns=["1", "2", "3", "4", "5", "6",]
type(player_list)

bs4.element.ResultSet

In [139]:
len(player_list)

29

In [154]:
def GetPlayerField(tr):
    cls = tr.attrs.get("class")
    if cls is not None and 'w_100p' in cls:
        img = tr.find("img")
        return img['title'] + '|' + img['src']
    return tr.text
l = []
for tr in player_list:
    td = tr.find_all('td')
    row = [GetPlayerField(tr) for tr in td]
    #print(row)
    l.append(row)

In [159]:
len(l[0])

3

In [162]:
if len(l[0]) == 6:
    IsFull = 1
    df_players = pd.DataFrame(l, columns=["Player", "Nation", "Amplua", "Games", "Goal", "Pass",])
elif len(l[0]) == 3:
    IsFull = 0
    df_players = pd.DataFrame(l, columns=["Player", "Nation", "Amplua"])

In [161]:
df_players.head(5)

,Player,Nation,Amplua
0,\nДжонатан Бонд\n,Англия|https://ss.sport-express.ru/img/flags/v...,\n\r\n В\r\n \n
1,\nСэм Джонстон\n,Англия|https://ss.sport-express.ru/img/flags/v...,\n\r\n В\r\n \n
2,\nБоаз Майхилл\n,Уэльс|https://ss.sport-express.ru/img/flags/v2...,\n\r\n В\r\n \n
3,\nАлекс Палмер\n,Англия|https://ss.sport-express.ru/img/flags/v...,\n\r\n В\r\n \n
4,\nСеми Аджайи\n,Нигерия|https://ss.sport-express.ru/img/flags/...,\n\r\n З\r\n \n


In [100]:
con = pyodbc.connect('Trusted_Connection=yes', driver = '{SQL Server}',server = 'localhost', port = '1433', database = 'SportStat', autocommit=True)

In [130]:
Team = 'Барселона'
IsOverwrite = 1
nations = {}
for index, row in df_players.iterrows():
    tst_dict = {}
    Player = row['Player'].strip()
    Amplua =  row['Amplua'].strip()
    if IsFull == 1:
        Games =  row['Games']
        Pass =  row['Pass']    
        if Amplua == 'В' and '(' in row['Goal']:
            Goals =  row['Goal'].split('(')[0]
            GoalsGK =  row['Goal'].split('(')[1].split(')')[0]
        else:
            Goals =  row['Goal']
            GoalsGK = 0
    else:
        Games = 0
        Pass = 0
        Goals = 0
        GoalsGK = 0
    tst_dict['URL'] = row['Nation'].split('|')[1]
    nations[row['Nation'].split('|')[0]] = tst_dict
    cursor = con.cursor()
    cursor.execute("exec pSaveLineUp ?,?,?,?,?,?,?,?", (Team, IsOverwrite, Player, Amplua, Games, Goals, Pass, GoalsGK))
    con.commit()

In [131]:
for nation_key, nation_value  in nations.items():
    response = req.get(nation_value['URL'])
    nations[nation_key]['Img'] = Image.open(BytesIO(response.content))

In [ ]:
nations

In [133]:
cursor = con.cursor()
for key, value in nations.items():
    output = io.BytesIO()
    value['Img'].save(output, format='PNG')
    cursor.execute("exec pSaveNation ?,?,?", (str(key), value['URL'], pyodbc.Binary(output.getvalue())))
con.commit()

In [163]:
nations = {}
def GetPlayerField(tr):
    cls = tr.attrs.get("class")
    if cls is not None and 'w_100p' in cls:
        img = tr.find("img")
        return img['title'] + '|' + img['src']
    return tr.text

def GetTeamlineUp(con, Team, url, IsOverwrite):
    resp = req.get(url) 
    soup_team = BeautifulSoup(resp.text, 'lxml')
    player_table = soup_team.find("table", {"class": "se19-table"})
    player_list = player_table.find('tbody').find_all('tr')
    
    l = []
    for tr in player_list:
        td = tr.find_all('td')
        row = [GetPlayerField(tr) for tr in td]
        #print(row)
        l.append(row)
    if len(l[0]) == 6:
        IsFull = 1
        df_players = pd.DataFrame(l, columns=["Player", "Nation", "Amplua", "Games", "Goal", "Pass",])
    elif len(l[0]) == 3:
        IsSimple = 1
        df_players = pd.DataFrame(l, columns=["Player", "Nation", "Amplua"])
    for index, row in df_players.iterrows():
        tst_dict = {}
        Player = row['Player'].strip()
        Amplua =  row['Amplua'].strip()
        if IsFull == 1:
            Games =  row['Games']
            Pass =  row['Pass']    
            if Amplua == 'В' and '(' in row['Goal']:
                Goals =  row['Goal'].split('(')[0]
                GoalsGK =  row['Goal'].split('(')[1].split(')')[0]
            else:
                Goals =  row['Goal']
                GoalsGK = 0
        else:
            Games = 0
            Pass = 0
            Goals = 0
            GoalsGK = 0
        tst_dict['URL'] = row['Nation'].split('|')[1]
        nations[row['Nation'].split('|')[0]] = tst_dict
        cursor = con.cursor()
        cursor.execute("exec pSaveLineUp ?,?,?,?,?,?,?,?", (Team, IsOverwrite, Player, Amplua, Games, Goals, Pass, GoalsGK))
        con.commit()
def SaveNations(con, nations):   
    for nation_key, nation_value  in nations.items():
        response = req.get(nation_value['URL'])
        nations[nation_key]['Img'] = Image.open(BytesIO(response.content))
    cursor = con.cursor()
    for key, value in nations.items():
        output = io.BytesIO()
        value['Img'].save(output, format='PNG')
        cursor.execute("exec pSaveNation ?,?,?", (str(key), value['URL'], pyodbc.Binary(output.getvalue())))
    con.commit()
    